In [ ]:
import pandas as pd
import numpy as np
import tushare as ts
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

code = '002397'
length = 10 #10日移动平均
open_trigger = 0.5 #价格向上偏离均线0.5倍观察期内标准差的最大值开仓
stopwin_trigger = 3 #价格向上偏离均线3倍观察期内标准差的最大值止盈
stoplose_trigger = 1 #移动止损：跌破均值移动止损；固定止损：开仓价向下偏离观察期内标准差的最大值

data = ts.get_k_data(code, '2012-01-01', '2017-01-01')
data['pct_change'] = data['close'].pct_change()
data['ma'] = data['close'].rolling(window=length, min_periods=3).mean()
data['std'] = data['close'].rolling(window=length, min_periods=3).std()

data['std_limit'] = data['std'].rolling(window=length).max() #观察期(10日移动平均)内标准差的最大值
data['yes_ma'] = data['ma'].shift(1)
data['yes_std_limit'] = data['std_limit'].shift(1)

data['long_open_price'] = data['yes_ma'] + data['yes_std_limit']*open_trigger #今日的开仓价为昨天10日移动平均+0.5倍观察期内标准差最大值
data['long_stopwin_price'] = data['yes_ma'] + data['yes_std_limit']*stopwin_trigger #止盈价为昨天10日移动平均+3倍观察期内标准差最大值

data['long_open_signal'] = np.where(data['high'] > data['long_open_price'], 1, 0) #如果当日最高价高于开仓价则开仓信号为1
data['long_stopwin_signal'] = np.where(data['high'] > data['long_stopwin_price'], 1, 0) #如果当日最高价高于止盈价则止盈信号为1，如果不高于则止盈信号为0

flag = 0 #记录持仓情况，0代表空仓，1代表持仓

for i in range(12, (len(data)-1)): #前10个数据为无效，然后要算昨天的观察期内标准差最大值所以第11个数据也不能用，只能从第12个数据开始。由于最后一天开仓有可能当天就会触发平仓但由于T+1只能再后一天卖出，会导致无数据问题，所以选择倒数第二个数据终止循环
    if flag == 1: #如果之前有持仓
        stoplose_price = max(data.loc[i,'yes_ma'], long_open_price - long_open_delta * stoplose_trigger) #计算止损价，取均线(移动止损)和开仓价下移一定倍数开仓时观察期标准差最大值(固定止损)中的最大值作为止损价
        if data.loc[i, 'long_stopwin_signal']: #如果当日最高价高于止盈价
            data.loc[i, 'return'] = data.loc[i, 'long_stopwin_price']/data.loc[i-1, 'close'] - 1 #计算以止盈价平仓的收益并平仓
            flag = 0

        elif data.loc[i, 'low'] < stoplose_price: #如果当日最低价低于止损价
            data.loc[i, 'return'] = min(data.loc[i, 'open'], stoplose_price)/data.loc[i-1, 'close'] - 1 #计算以止损价和开盘价的最小值平仓的收益并平仓(防止开盘价低于止损价且不上涨无法以止损价卖出的情况)
            flag = 0

        else: #如果没有触发止盈和止损信号
            data.loc[i, 'return'] = data.loc[i, 'close']/data.loc[i-1, 'close'] - 1 #用收盘价计算当日收益率并继续持仓

    else: #如果之前无持仓
        if data.loc[i, 'long_open_signal']: #如果当日开仓信号为1
            flag = 1
            long_open_price = max(data.loc[i, 'open'], data.loc[i, 'long_open_price']) #计算实际买入价格(开盘价与开仓价取最大值，防止开盘价高于开仓价且不下跌无法以开仓价买入的情况)
            long_open_delta = data.loc[i, 'yes_std_limit'] #提取当日前一天的观察期内标准差最大值
            data.loc[i, 'return'] = data.loc[i, 'close']/long_open_price - 1 #计算当日收益率

            stoplose_price = max(data.loc[i,'yes_ma'], long_open_price - long_open_delta * stoplose_trigger) #计算止损价，取均线(移动止损)和开仓价下移一定倍数开仓时观察期标准差最大值(固定止损)中的最大值作为止损价
            if (data.loc[i, 'low'] < stoplose_price or data.loc[i, 'long_stopwin_signal']): #如果当天开仓但在日内触发了止盈或止损
                data.loc[i, 'return'] = data.loc[i+1, 'open']/long_open_price - 1 #计算以第二天开盘价平仓的收益并平仓
                flag = 0

data['return'].fillna(0, inplace=True)
data['strategy_return'] = (data['return'] + 1).cumprod() #计算累计收益
data['stock_return'] = (data['pct_change'] + 1).cumprod()

matplotlib.style.use('ggplot')
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)
ax.plot(data.stock_return)
ax.plot(data.strategy_return)
plt.title(code)
plt.legend()
plt.show()